<a href="https://colab.research.google.com/github/hrishikesht7/CropRecommendationSystem/blob/main/CropRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("atharvaingle/crop-recommendation-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'crop-recommendation-dataset' dataset.
Path to dataset files: /kaggle/input/crop-recommendation-dataset


In [3]:
!pip install sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [5]:
from sklearn import preprocessing

In [10]:
pip install kagglehub pandas numpy scikit-learn torch xgboost lightgbm joblib seaborn matplotlib

In [15]:
import kagglehub
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, mutual_info_classif
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier, VotingClassifier
import joblib
from scipy.stats import randint, uniform
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

# 1. Load and preprocess
path = kagglehub.dataset_download("atharvaingle/crop-recommendation-dataset")
print("Path to dataset files:", path)

data = pd.read_csv(f"{path}/Crop_recommendation.csv")
X = data.drop('label', axis=1)
y = data['label']

# Polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_poly = poly.fit_transform(X)
X = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(X.columns))

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Feature selection
selector = SelectKBest(score_func=mutual_info_classif, k=20)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Keep feature names for LightGBM
selected_features = X_train.columns[selector.get_support()].tolist()
X_train_selected = pd.DataFrame(X_train_selected, columns=selected_features)
X_test_selected = pd.DataFrame(X_test_selected, columns=selected_features)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Save preprocessors
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')
joblib.dump(poly, 'poly_features.pkl')
joblib.dump(selector, 'feature_selector.pkl')

print(f"Features after selection: {X_train_scaled.shape[1]}")

input_dim = X_train_scaled.shape[1]
num_classes = len(label_encoder.classes_)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert to tensors
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_torch = torch.tensor(y_test, dtype=torch.long).to(device)

# 2. LeNet-inspired
class LeNetInspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(LeNetInspiredModel, self).__init__()
        self.reshape = lambda x: x.view(-1, input_dim, 1)
        self.conv1 = nn.Conv1d(input_dim, 16, kernel_size=5, padding=2)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(32, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.pool = nn.MaxPool1d(1)  # Minimal pooling for tabular data

    def forward(self, x):
        x = self.reshape(x)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x)

lenet_model = LeNetInspiredModel(input_dim, num_classes).to(device)

# 3. GoogLeNet-inspired
class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionModule, self).__init__()
        self.branch1 = nn.Conv1d(in_channels, out_channels // 4, kernel_size=1)
        self.branch2 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels // 4, kernel_size=1),
            nn.Conv1d(out_channels // 4, out_channels // 4, kernel_size=3, padding=1)
        )
        self.branch3 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels // 4, kernel_size=1),
            nn.Conv1d(out_channels // 4, out_channels // 4, kernel_size=5, padding=2)
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool1d(kernel_size=1),
            nn.Conv1d(in_channels, out_channels // 4, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        return torch.cat([branch1, branch2, branch3, branch4], dim=1)

class GoogLeNetInspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(GoogLeNetInspiredModel, self).__init__()
        self.reshape = lambda x: x.view(-1, input_dim, 1)
        self.conv1 = nn.Conv1d(input_dim, 64, kernel_size=3, padding=1)
        self.inception1 = InceptionModule(64, 128)
        self.inception2 = InceptionModule(128, 128)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = self.reshape(x)
        x = F.relu(self.conv1(x))
        x = self.inception1(x)
        x = self.inception2(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

googlenet_model = GoogLeNetInspiredModel(input_dim, num_classes).to(device)

# 4. AlexNet-inspired
class AlexNetInspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(AlexNetInspiredModel, self).__init__()
        self.reshape = lambda x: x.view(-1, input_dim, 1)
        self.conv1 = nn.Conv1d(input_dim, 96, kernel_size=11, stride=1, padding=5)
        self.conv2 = nn.Conv1d(96, 256, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(256, 384)
        self.fc2 = nn.Linear(384, 192)
        self.fc3 = nn.Linear(192, num_classes)
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool1d(1)

    def forward(self, x):
        x = self.reshape(x)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x)

alexnet_model = AlexNetInspiredModel(input_dim, num_classes).to(device)

# 5. Channel-Boosted CNN
class ChannelBoostedCNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ChannelBoostedCNN, self).__init__()
        self.reshape = lambda x: x.view(-1, input_dim, 1)
        self.conv1 = nn.Conv1d(input_dim, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(256, 512, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.pool = nn.MaxPool1d(1)

    def forward(self, x):
        x = self.reshape(x)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

channel_boosted_model = ChannelBoostedCNN(input_dim, num_classes).to(device)

# 6. Inception V2-inspired
class InceptionV2Module(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionV2Module, self).__init__()
        self.branch1 = nn.Conv1d(in_channels, out_channels // 4, kernel_size=1)
        self.branch2 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels // 8, kernel_size=1),
            nn.Conv1d(out_channels // 8, out_channels // 4, kernel_size=3, padding=1)
        )
        self.branch3 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels // 8, kernel_size=1),
            nn.Conv1d(out_channels // 8, out_channels // 4, kernel_size=3, padding=1),
            nn.Conv1d(out_channels // 4, out_channels // 4, kernel_size=3, padding=1)
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool1d(kernel_size=1),
            nn.Conv1d(in_channels, out_channels // 4, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        return torch.cat([branch1, branch2, branch3, branch4], dim=1)

class InceptionV2InspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(InceptionV2InspiredModel, self).__init__()
        self.reshape = lambda x: x.view(-1, input_dim, 1)
        self.conv1 = nn.Conv1d(input_dim, 64, kernel_size=3, padding=1)
        self.inception1 = InceptionV2Module(64, 128)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = self.reshape(x)
        x = F.relu(self.conv1(x))
        x = self.inception1(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

inception_v2_model = InceptionV2InspiredModel(input_dim, num_classes).to(device)

# 7. Inception V3-inspired
class InceptionV3InspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(InceptionV3InspiredModel, self).__init__()
        self.reshape = lambda x: x.view(-1, input_dim, 1)
        self.conv1 = nn.Conv1d(input_dim, 64, kernel_size=3, padding=1)
        self.inception1 = InceptionV2Module(64, 128)
        self.inception2 = InceptionV2Module(128, 256)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = self.reshape(x)
        x = F.relu(self.conv1(x))
        x = self.inception1(x)
        x = self.inception2(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

inception_v3_model = InceptionV3InspiredModel(input_dim, num_classes).to(device)

# 8. Inception V4-inspired
class InceptionV4InspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(InceptionV4InspiredModel, self).__init__()
        self.reshape = lambda x: x.view(-1, input_dim, 1)
        self.conv1 = nn.Conv1d(input_dim, 64, kernel_size=3, padding=1)
        self.inception1 = InceptionV2Module(64, 128)
        self.inception2 = InceptionV2Module(128, 256)
        self.inception3 = InceptionV2Module(256, 384)
        self.fc1 = nn.Linear(384, 192)
        self.fc2 = nn.Linear(192, num_classes)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = self.reshape(x)
        x = F.relu(self.conv1(x))
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.inception3(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

inception_v4_model = InceptionV4InspiredModel(input_dim, num_classes).to(device)

# 9. ResNeXt-inspired
class ResNeXtBlock(nn.Module):
    def __init__(self, in_channels, out_channels, cardinality=4):
        super(ResNeXtBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1, groups=cardinality)
        self.conv3 = nn.Conv1d(out_channels, out_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.bn3 = nn.BatchNorm1d(out_channels)
        self.shortcut = nn.Conv1d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else nn.Identity()

    def forward(self, x):
        shortcut = self.shortcut(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = x + shortcut
        return F.relu(x)

class ResNeXtInspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ResNeXtInspiredModel, self).__init__()
        self.reshape = lambda x: x.view(-1, input_dim, 1)
        self.conv1 = nn.Conv1d(input_dim, 64, kernel_size=3, padding=1)
        self.block1 = ResNeXtBlock(64, 128)
        self.block2 = ResNeXtBlock(128, 128)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = self.reshape(x)
        x = F.relu(self.conv1(x))
        x = self.block1(x)
        x = self.block2(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

resnext_model = ResNeXtInspiredModel(input_dim, num_classes).to(device)

# 10. DenseNet-inspired
class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(DenseBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, growth_rate, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(growth_rate)
        self.conv2 = nn.Conv1d(in_channels + growth_rate, growth_rate, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(growth_rate)

    def forward(self, x):
        out1 = F.relu(self.bn1(self.conv1(x)))
        out = torch.cat([x, out1], dim=1)
        out2 = F.relu(self.bn2(self.conv2(out)))
        return torch.cat([x, out1, out2], dim=1)

class DenseNetInspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(DenseNetInspiredModel, self).__init__()
        self.reshape = lambda x: x.view(-1, input_dim, 1)
        self.conv1 = nn.Conv1d(input_dim, 64, kernel_size=3, padding=1)
        self.dense1 = DenseBlock(64, 32)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = self.reshape(x)
        x = F.relu(self.conv1(x))
        x = self.dense1(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

densenet_model = DenseNetInspiredModel(input_dim, num_classes).to(device)

# 11. VGG-inspired
class VGGInspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(VGGInspiredModel, self).__init__()
        self.reshape = lambda x: x.view(-1, input_dim, 1)
        self.conv1 = nn.Conv1d(input_dim, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(128, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool1d(1)

    def forward(self, x):
        x = self.reshape(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

vgg_model = VGGInspiredModel(input_dim, num_classes).to(device)

# 12. Transformer-inspired
class TransformerInspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=128, num_heads=4, num_layers=2):
        super(TransformerInspiredModel, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.pos_embed = nn.Parameter(torch.randn(1, 1, hidden_dim))
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1) + self.pos_embed
        x = self.transformer(x)
        x = x.squeeze(1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

transformer_model = TransformerInspiredModel(input_dim, num_classes).to(device)

# 13. EfficientNet-inspired
class EfficientNetInspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EfficientNetInspiredModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.attention = nn.MultiheadAttention(embed_dim=128, num_heads=4, batch_first=True)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.swish = nn.SiLU()

    def forward(self, x):
        x = self.swish(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.swish(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        attn_output, _ = self.attention(x.unsqueeze(1), x.unsqueeze(1), x.unsqueeze(1))
        x = x + attn_output.squeeze(1)
        x = F.layer_norm(x, x.shape[1:])
        x = self.swish(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        return self.fc4(x)

effnet_model = EfficientNetInspiredModel(input_dim, num_classes).to(device)

# 14. ConvNeXt-inspired
class ConvNeXtInspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ConvNeXtInspiredModel, self).__init__()
        self.reshape = lambda x: x.view(-1, input_dim, 1)
        self.conv1 = nn.Conv1d(input_dim, 40, kernel_size=7, padding=3, groups=input_dim)
        self.conv2 = nn.Conv1d(40, 40, kernel_size=1)
        self.ln1 = nn.LayerNorm([40, 1])
        self.fc1 = nn.Linear(40, 64)
        self.ln2 = nn.LayerNorm(64)
        self.fc2 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.swish = nn.SiLU()

    def forward(self, x):
        x = self.reshape(x)
        x = self.swish(self.conv1(x))
        x = self.swish(self.conv2(x))
        x = self.ln1(x).squeeze(2)
        x = self.swish(self.fc1(x))
        x = self.ln2(x)
        x = self.dropout(x)
        return self.fc2(x)

convnext_model = ConvNeXtInspiredModel(input_dim, num_classes).to(device)

# 15. ViT-inspired
class ViTInspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes, num_heads=8, hidden_dim=128, num_patches=4):
        super(ViTInspiredModel, self).__init__()
        patch_dim = input_dim // num_patches
        self.patch_embed = nn.Linear(patch_dim, hidden_dim)
        self.pos_embed = nn.Parameter(torch.randn(1, num_patches, hidden_dim))
        self.attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, hidden_dim))

    def forward(self, x):
        batch_size = x.shape[0]
        x_patched = x.view(batch_size, -1, self.patch_embed.in_features)
        x_embed = self.patch_embed(x_patched)
        x_embed += self.pos_embed
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat([cls_tokens, x_embed], dim=1)
        attn_output, _ = self.attention(x, x, x)
        x = x + attn_output
        x = self.layer_norm(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = x[:, 0]
        return self.fc2(x)

vit_model = ViTInspiredModel(input_dim, num_classes).to(device)

# 16. ResNet-inspired
class ResidualBlock(nn.Module):
    def __init__(self, in_units, out_units):
        super(ResidualBlock, self).__init__()
        self.fc1 = nn.Linear(in_units, out_units)
        self.bn1 = nn.BatchNorm1d(out_units)
        self.fc2 = nn.Linear(out_units, out_units)
        self.bn2 = nn.BatchNorm1d(out_units)
        self.dropout = nn.Dropout(0.3)
        self.swish = nn.SiLU()
        self.shortcut = nn.Linear(in_units, out_units) if in_units != out_units else nn.Identity()

    def forward(self, x):
        shortcut = self.shortcut(x)
        x = self.swish(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.bn2(self.fc2(x))
        x = x + shortcut
        return self.swish(x)

class ResNetInspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ResNetInspiredModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.res_block1 = ResidualBlock(256, 128)
        self.res_block2 = ResidualBlock(128, 128)
        self.res_block3 = ResidualBlock(128, 64)
        self.res_block4 = ResidualBlock(64, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.5)
        self.swish = nn.SiLU()

    def forward(self, x):
        x = self.swish(self.bn1(self.fc1(x)))
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.res_block4(x)
        x = self.dropout(x)
        return self.fc2(x)

resnet_model = ResNetInspiredModel(input_dim, num_classes).to(device)

# 17. TabNet-inspired
class TabNetInspiredModel(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=128, n_steps=3):
        super(TabNetInspiredModel, self).__init__()
        self.feature_transformer = nn.Linear(input_dim, hidden_dim)
        self.attention = nn.ModuleList([
            nn.Linear(hidden_dim, hidden_dim) for _ in range(n_steps)
        ])
        self.feature_blocks = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.3)
            ) for _ in range(n_steps)
        ])
        self.final_fc = nn.Linear(hidden_dim, num_classes)
        self.layer_norm = nn.LayerNorm(hidden_dim)

    def forward(self, x):
        x = self.feature_transformer(x)
        for attn, block in zip(self.attention, self.feature_blocks):
            attn_weights = F.softmax(attn(x), dim=1)
            x = x * attn_weights
            x = block(x)
            x = self.layer_norm(x)
        return self.final_fc(x)

tabnet_model = TabNetInspiredModel(input_dim, num_classes).to(device)

# Training function with loss tracking
def train_model(model, X_train, y_train, X_val, y_val, epochs=100, lr=0.001):
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    criterion = nn.CrossEntropyLoss()
    best_loss = float('inf')
    patience = 10
    counter = 0
    train_losses = []
    val_losses = []

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        scheduler.step()

        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val)
        model.train()

        train_losses.append(loss.item())
        val_losses.append(val_loss.item())

        if (epoch + 1) % 10 == 0:
            print(f"{model.__class__.__name__} Epoch {epoch + 1}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

        if val_loss < best_loss:
            best_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), f'best_{model.__class__.__name__}.pth')
        else:
            counter += 1
            if counter >= patience:
                break

    model.load_state_dict(torch.load(f'best_{model.__class__.__name__}.pth'))
    return model, train_losses, val_losses

# Train all models and collect losses
models = {
    'lenet': lenet_model,
    'googlenet': googlenet_model,
    'alexnet': alexnet_model,
    'channel_boosted': channel_boosted_model,
    'inception_v2': inception_v2_model,
    'inception_v3': inception_v3_model,
    'inception_v4': inception_v4_model,
    'resnext': resnext_model,
    'densenet': densenet_model,
    'vgg': vgg_model,
    'transformer': transformer_model,
    'effnet': effnet_model,
    'convnext': convnext_model,
    'vit': vit_model,
    'resnet': resnet_model,
    'tabnet': tabnet_model
}

accuracies = {}
loss_curves = {}
for name, model in models.items():
    model, train_losses, val_losses = train_model(model, X_train_torch, y_train_torch, X_test_torch, y_test_torch)
    loss_curves[name] = (train_losses, val_losses)
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_torch)
        _, pred_classes = torch.max(outputs, 1)
        acc = accuracy_score(y_test, pred_classes.cpu().numpy())
        accuracies[name] = acc
        print(f"{name.capitalize()} Accuracy: {acc:.4f}")
        print(classification_report(y_test, pred_classes.cpu().numpy(), target_names=label_encoder.classes_))
    torch.save(model.state_dict(), f'{name}_model.pth')

# 18. Ensemble with XGBoost/LightGBM
xgb_model = XGBClassifier(random_state=42)
lgb_model = LGBMClassifier(random_state=42, verbose=-1)

xgb_params = {'n_estimators': randint(100, 500), 'max_depth': randint(3, 10), 'learning_rate': uniform(0.01, 0.3)}
xgb_tuned = RandomizedSearchCV(xgb_model, xgb_params, n_iter=10, cv=5, random_state=42)
xgb_tuned.fit(X_train_selected, y_train)

stacking_model = StackingClassifier(estimators=[('xgb', xgb_tuned.best_estimator_), ('lgb', lgb_model)],
                                    final_estimator=XGBClassifier(random_state=42))
stacking_model.fit(X_train_selected, y_train)
stacking_pred = stacking_model.predict(X_test_selected)
stacking_acc = accuracy_score(y_test, stacking_pred)
print(f"Stacking Ensemble Accuracy: {stacking_acc:.4f}")
print(classification_report(y_test, stacking_pred, target_names=label_encoder.classes_))

joblib.dump(stacking_model, 'stacking_model.pkl')

# 19. Final Soft Voting Ensemble
models_dict = {
    'lenet': (lambda x: torch.softmax(lenet_model(x), dim=1).cpu().detach().numpy(), accuracies['lenet']),
    'googlenet': (lambda x: torch.softmax(googlenet_model(x), dim=1).cpu().detach().numpy(), accuracies['googlenet']),
    'alexnet': (lambda x: torch.softmax(alexnet_model(x), dim=1).cpu().detach().numpy(), accuracies['alexnet']),
    'channel_boosted': (lambda x: torch.softmax(channel_boosted_model(x), dim=1).cpu().detach().numpy(), accuracies['channel_boosted']),
    'inception_v2': (lambda x: torch.softmax(inception_v2_model(x), dim=1).cpu().detach().numpy(), accuracies['inception_v2']),
    'inception_v3': (lambda x: torch.softmax(inception_v3_model(x), dim=1).cpu().detach().numpy(), accuracies['inception_v3']),
    'inception_v4': (lambda x: torch.softmax(inception_v4_model(x), dim=1).cpu().detach().numpy(), accuracies['inception_v4']),
    'resnext': (lambda x: torch.softmax(resnext_model(x), dim=1).cpu().detach().numpy(), accuracies['resnext']),
    'densenet': (lambda x: torch.softmax(densenet_model(x), dim=1).cpu().detach().numpy(), accuracies['densenet']),
    'vgg': (lambda x: torch.softmax(vgg_model(x), dim=1).cpu().detach().numpy(), accuracies['vgg']),
    'transformer': (lambda x: torch.softmax(transformer_model(x), dim=1).cpu().detach().numpy(), accuracies['transformer']),
    'effnet': (lambda x: torch.softmax(effnet_model(x), dim=1).cpu().detach().numpy(), accuracies['effnet']),
    'convnext': (lambda x: torch.softmax(convnext_model(x), dim=1).cpu().detach().numpy(), accuracies['convnext']),
    'vit': (lambda x: torch.softmax(vit_model(x), dim=1).cpu().detach().numpy(), accuracies['vit']),
    'resnet': (lambda x: torch.softmax(resnet_model(x), dim=1).cpu().detach().numpy(), accuracies['resnet']),
    'tabnet': (lambda x: torch.softmax(tabnet_model(x), dim=1).cpu().detach().numpy(), accuracies['tabnet']),
    'stacking': (lambda x: stacking_model.predict_proba(x), stacking_acc)
}

total_acc = sum(acc for _, acc in models_dict.values())
weights = {name: acc / total_acc for name, (pred_fn, acc) in models_dict.items()}

ensemble_probs = np.zeros((len(X_test_scaled), num_classes))
for name, (pred_fn, _) in models_dict.items():
    if name == 'stacking':
        probs = pred_fn(X_test_selected)
    else:
        probs = pred_fn(X_test_torch)
    ensemble_probs += weights[name] * probs

final_pred_classes = np.argmax(ensemble_probs, axis=1)
final_acc = accuracy_score(y_test, final_pred_classes)
print(f"Final Soft Ensemble Accuracy: {final_acc:.4f}")
print(classification_report(y_test, final_pred_classes, target_names=label_encoder.classes_))

# Cross-validation
cv_scores = cross_val_score(VotingClassifier(estimators=[('xgb', xgb_tuned.best_estimator_), ('lgb', lgb_model)], voting='soft'),
                            X_train_selected, y_train, cv=5)
print(f"CV Mean Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# 20. Visualizations with Matplotlib
plt.style.use('default')  # Use default Matplotlib style

# (a) Confusion Matrix
plt.figure(figsize=(12, 10))
cm_matrix = confusion_matrix(y_test, final_pred_classes)
plt.imshow(cm_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()
plt.title('Confusion Matrix - Final Ensemble')
plt.xlabel('Predicted')
plt.ylabel('True')
tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, label_encoder.classes_, rotation=45, ha='right')
plt.yticks(tick_marks, label_encoder.classes_)
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j, i, cm_matrix[i, j], ha='center', va='center', color='black' if cm_matrix[i, j] < cm_matrix.max() / 2 else 'white')
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.close()

# (b) Feature Importance Bar Plot
mi_scores = mutual_info_classif(X_train, y_train)
feature_names = X_train.columns
mi_df = pd.DataFrame({'Feature': feature_names, 'MI Score': mi_scores})
mi_df = mi_df.sort_values('MI Score', ascending=False).head(10)
plt.figure(figsize=(10, 6))
plt.barh(mi_df['Feature'], mi_df['MI Score'], color=plt.cm.viridis(np.linspace(0, 1, len(mi_df))))
plt.title('Top 10 Feature Importance (Mutual Information)')
plt.xlabel('Mutual Information Score')
plt.ylabel('Feature')
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()

# (c) Model Accuracy Bar Plot
acc_df = pd.DataFrame({
    'Model': list(accuracies.keys()) + ['stacking', 'ensemble'],
    'Accuracy': list(accuracies.values()) + [stacking_acc, final_acc]
})
plt.figure(figsize=(12, 6))
plt.barh(acc_df['Model'], acc_df['Accuracy'], color=plt.cm.magma(np.linspace(0, 1, len(acc_df))))
plt.title('Model Accuracy Comparison')
plt.xlabel('Accuracy')
plt.ylabel('Model')
plt.xlim(0.95, 1.0)
plt.tight_layout()
plt.savefig('model_accuracy.png')
plt.close()

# (d) Training Loss Curves
plt.figure(figsize=(12, 6))
for name, (train_losses, val_losses) in loss_curves.items():
    epochs = range(1, len(train_losses) + 1)
    plt.plot(epochs, train_losses, label=f'{name.capitalize()} Train', linestyle='-')
    plt.plot(epochs, val_losses, label=f'{name.capitalize()} Val', linestyle='--')
plt.title('Training and Validation Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.savefig('loss_curves.png')
plt.close()

# (e) Per-Class F1 Scores
f1_scores = f1_score(y_test, final_pred_classes, average=None)
f1_df = pd.DataFrame({
    'Class': label_encoder.classes_,
    'F1 Score': f1_scores
})
plt.figure(figsize=(12, 6))
plt.barh(f1_df['Class'], f1_df['F1 Score'], color=plt.cm.coolwarm(np.linspace(0, 1, len(f1_df))))
plt.title('Per-Class F1 Scores - Final Ensemble')
plt.xlabel('F1 Score')
plt.ylabel('Class')
plt.tight_layout()
plt.savefig('f1_scores.png')
plt.close()

# 21. Prediction function (Real-time input and example)
def predict_crop(N=None, P=None, K=None, temperature=None, humidity=None, pH=None, rainfall=None):
    if N is None:  # Interactive input
        print("Enter values for crop prediction:")
        N = float(input("Nitrogen (N): "))
        P = float(input("Phosphorus (P): "))
        K = float(input("Potassium (K): "))
        temperature = float(input("Temperature (°C): "))
        humidity = float(input("Humidity (%): "))
        pH = float(input("pH: "))
        rainfall = float(input("Rainfall (mm): "))

    input_data = np.array([[N, P, K, temperature, humidity, pH, rainfall]])
    input_poly = poly.transform(input_data)
    input_selected = selector.transform(input_poly)
    input_selected_df = pd.DataFrame(input_selected, columns=selected_features)
    input_scaled = scaler.transform(input_selected_df)
    input_torch = torch.tensor(input_scaled, dtype=torch.float32).to(device)

    probs = {}
    for name, (pred_fn, _) in models_dict.items():
        if name == 'stacking':
            probs[name] = pred_fn(input_selected_df)
        else:
            with torch.no_grad():
                probs[name] = pred_fn(input_torch)  # Already softmaxed in models_dict

    ensemble_prob = np.zeros((1, num_classes))  # Shape (1, 22) for single sample
    for name, prob in probs.items():
        ensemble_prob += weights[name] * prob

    final_class = np.argmax(ensemble_prob, axis=1)
    final_crop = label_encoder.inverse_transform(final_class)[0]
    individual_preds = [label_encoder.inverse_transform([np.argmax(prob)])[0] for prob in probs.values()]
    return final_crop, individual_preds

# Example prediction
example_input = [90, 42, 43, 20.879744, 82.002744, 6.502985, 202.935536]
predicted_crop, individual_preds = predict_crop(*example_input)
print(f"Example Prediction - Recommended Crop: {predicted_crop}")
print(f"Example Prediction - Individual Model Predictions: {individual_preds}")

# Real-time prediction
predicted_crop, individual_preds = predict_crop()
print(f"Real-Time Prediction - Recommended Crop: {predicted_crop}")
print(f"Real-Time Prediction - Individual Model Predictions: {individual_preds}")

print("Visualizations saved: confusion_matrix.png, feature_importance.png, model_accuracy.png, loss_curves.png, f1_scores.png")

Using Colab cache for faster access to the 'crop-recommendation-dataset' dataset.
Path to dataset files: /kaggle/input/crop-recommendation-dataset
Features after selection: 20
LeNetInspiredModel Epoch 10, Loss: 3.0833, Val Loss: 3.0870
LeNetInspiredModel Epoch 20, Loss: 3.0621, Val Loss: 3.0624
LeNetInspiredModel Epoch 30, Loss: 2.9961, Val Loss: 2.9927
LeNetInspiredModel Epoch 40, Loss: 2.8707, Val Loss: 2.8514
LeNetInspiredModel Epoch 50, Loss: 2.6673, Val Loss: 2.6458
LeNetInspiredModel Epoch 60, Loss: 2.4760, Val Loss: 2.4543
LeNetInspiredModel Epoch 70, Loss: 2.3568, Val Loss: 2.3355
LeNetInspiredModel Epoch 80, Loss: 2.2939, Val Loss: 2.2750
LeNetInspiredModel Epoch 90, Loss: 2.2747, Val Loss: 2.2524
LeNetInspiredModel Epoch 100, Loss: 2.2657, Val Loss: 2.2490
Lenet Accuracy: 0.2250
              precision    recall  f1-score   support

       apple       0.61      1.00      0.75        23
      banana       0.00      0.00      0.00        21
   blackgram       0.00      0.00    